<a href="https://colab.research.google.com/github/Saputoa21/ADS_2024_Saputoa/blob/master/exercises/HomeExercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [3]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")
print(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [5]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first 50 tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 192)).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [6]:
small_imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 128
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
})

In [7]:
small_imdb_dataset["train"]["text"][:10]

["I didn't know this was a silent movie with narration. I don't care for silent movies - the corny humor, flickering lighting and film, etc. I'm sure that attributes to the low score I assigned it. It was about chapter 8 before I found any interest in this story and",
 'Well, first off, if you\'re checking out Revolt of the Zombies as some very early Night of the Living Dead (1968)-type film, forget it. This is about "zombies" in a more psychological sense, where that term merely denotes someone who is not in control of their will, but who must',
 'John Thaw, of Inspector Morse fame, plays old Tom Oakley in this movie. Tom lives in a tiny English village during 1939 and the start of the Second World War. A bit of a recluse, Tom has not yet recovered from the death of his wife and son while he',
 'I view probably 200 movies a year both at theaters and at home and I can say with confidence that this movie is by far the worst I have seen this year (If not ever, however I have not actually

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

Original

In [8]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=5,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.693300,0.698454,0.468750
2,0.691100,0.695595,0.468750
3,0.678000,0.690013,0.468750
4,0.671200,0.682187,0.562500
5,0.648700,0.679712,0.593750


TrainOutput(global_step=40, training_loss=0.6764307737350463, metrics={'train_runtime': 471.6528, 'train_samples_per_second': 1.357, 'train_steps_per_second': 0.085, 'total_flos': 18636507148416.0, 'train_loss': 0.6764307737350463, 'epoch': 5.0})

In [10]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[-0.17738849, -0.1273813 ],
       [-0.21873909, -0.02648006],
       [-0.21151853, -0.07537212],
       [-0.23246783, -0.07830766],
       [-0.17508106, -0.12822151],
       [-0.18223268, -0.11410393],
       [-0.16019382, -0.08181449],
       [-0.12709782, -0.17791714],
       [-0.18559942, -0.11994845],
       [-0.18621469, -0.13810745],
       [-0.13321853, -0.07785042],
       [-0.13303617, -0.15878068],
       [-0.14261247, -0.1389474 ],
       [-0.18185402, -0.11011443],
       [-0.22042562, -0.06310154],
       [-0.20142946, -0.11373382],
       [-0.1814025 , -0.09853655],
       [-0.17943585, -0.09252751],
       [-0.2332583 , -0.06660335],
       [-0.18777888, -0.15760595],
       [-0.21758425, -0.09412011],
       [-0.130912  , -0.11760622],
       [-0.14028274, -0.116117  ],
       [-0.14682189, -0.17974976],
       [-0.23324601, -0.02354055],
       [-0.15157063, -0.16541892],
       [-0.15144005, -0.1446986 ],
       [-0.20960654, -0.05

In [11]:
import torch

In [12]:
test_str = "I love this movie!"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-32")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

POSITIVE


In [15]:
test_set_string = small_imdb_dataset["test"]["text"][0]

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-32")
model_inputs = tokenizer(test_set_string, return_tensors="pt", padding=True, truncation=True)
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

NEGATIVE


In [16]:
test_set_results = trainer.predict(small_tokenized_dataset['test'])
print(test_set_results)

PredictionOutput(predictions=array([[-0.12628499, -0.1814078 ],
       [-0.13654362, -0.16797242],
       [-0.1676146 , -0.12522896],
       [-0.1624929 , -0.08154882],
       [-0.15949064, -0.15483914],
       [-0.2699575 , -0.04173411],
       [-0.20386572, -0.01026182],
       [-0.2156885 , -0.05279515],
       [-0.25099704, -0.0090025 ],
       [-0.19909038, -0.10893662],
       [-0.22393948, -0.10907632],
       [-0.22485451, -0.00032955],
       [-0.21493222, -0.07440689],
       [-0.1967145 , -0.05077852],
       [-0.22332624, -0.06344146],
       [-0.26871625, -0.04528964],
       [-0.2432943 , -0.05067471],
       [-0.26071727, -0.00381552],
       [-0.19610348, -0.11476539],
       [-0.19854954, -0.10746909],
       [-0.16860248, -0.0921507 ],
       [-0.1962165 , -0.08704866],
       [-0.13990955, -0.17649779],
       [-0.1305032 , -0.17894071],
       [-0.20959505, -0.06493679],
       [-0.2210635 , -0.05317314],
       [-0.15122974, -0.17080607],
       [-0.22553392, -0.04

In [17]:
test_trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [18]:
test_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.606800,0.636106,0.656250
2,0.498700,0.587420,0.718750
3,0.387000,0.559040,0.687500
4,0.303700,0.549808,0.718750
5,0.240700,0.537601,0.718750


TrainOutput(global_step=40, training_loss=0.40737428665161135, metrics={'train_runtime': 447.7848, 'train_samples_per_second': 1.429, 'train_steps_per_second': 0.089, 'total_flos': 18636507148416.0, 'train_loss': 0.40737428665161135, 'epoch': 5.0})

In [24]:
test_set = small_imdb_dataset["test"]["text"][0]

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-32")
model_inputs = tokenizer(test_set, return_tensors="pt", padding=True, truncation=True)
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

NEGATIVE


In [25]:
test_set = small_imdb_dataset["test"]["text"]

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-32")
tokenizer = AutoTokenizer.from_pretrained("sample_cl_trainer/checkpoint-32")

tokenized_test = tokenizer(
    test_set,  # All test sentences
    return_tensors="pt",  # PyTorch tensors
    padding=True,  # Pad to the longest sequence
    truncation=True,  # Truncate sequences longer than the model's max length
    max_length=128  # Optional: limit to the model's max token limit
)

import torch
from torch.utils.data import DataLoader, TensorDataset

# Create a DataLoader for batching
batch_size = 16  # Set an appropriate batch size
test_dataset = TensorDataset(tokenized_test["input_ids"], tokenized_test["attention_mask"])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

# Make predictions
predictions = []
fine_tuned_model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation for testing
    for batch in test_loader:
        input_ids, attention_mask = [t.to(device) for t in batch]
        logits = fine_tuned_model(input_ids=input_ids, attention_mask=attention_mask).logits
        batch_predictions = torch.argmax(logits, dim=1)  # Get predicted class indices
        predictions.extend(batch_predictions.cpu().numpy())

label_map = ["NEGATIVE", "POSITIVE"]

# Map predictions to labels
mapped_predictions = [label_map[p] for p in predictions]

# Print sample predictions
print(mapped_predictions[:10])

['NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE']


Modified

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first 50 tokens for speed on CPU
def truncate100(example):
    return {
        'text': " ".join(example['text'].split()[:100]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate100),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate100),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 170)).map(truncate100)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments2 = TrainingArguments(
    output_dir="sample_cl_trainer",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=16,
    num_train_epochs=5,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments2,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [ ]:
test_str = "I love this movie!"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-40")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

OSError: sample_cl_trainer/checkpoint-40 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`